# Classification of the Feature Neurons from the SNN
This notebook performs the classification of the feature neurons from the SNN according to certain criteria. In the future, it would be interesting to implement a 3rd layer in the SNN to classify the feature neurons automatically. For now, this manual classification will classify each neuron as one of the following:
- **Silent Neuron**: Neuron that does not fire at all.
- **Noisy Neuron**: Neuron that fires randomly or without a relevant pattern.
- **Ripple Neuron**: Neuron that fires in the presence of a ripple.
- **Fast Ripple Neuron**: Neuron that fires in the presence of a fast ripple.

## Check WD (change if necessary) and file loading

In [31]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/snn" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/snn"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn


## Load the Voltage and Current Dynamics during the SNN run

In [32]:
import numpy as np
from utils.io import preview_np_array
from utils.input import MarkerType, band_to_file_name

# Declare if using ripples, fast ripples, or both
chosen_band = MarkerType.FAST_RIPPLE     # RIPPLE, FAST_RIPPLE, or BOTH
band_file_name = band_to_file_name(chosen_band)

INPUT_PATH = f"./results/custom_subset_90-119_segment500_200"
TIME_SUFFIX = "time900-6000-1"

# Load the voltage and current data from the numpy files
voltage_file_name = f"{INPUT_PATH}/{band_file_name}_v_dynamics_0.07dv_5ch_{TIME_SUFFIX}.npy"
current_file_name = f"{INPUT_PATH}/{band_file_name}_u_dynamics_0.07dv_5ch_{TIME_SUFFIX}.npy"

v_dynamics = np.load(voltage_file_name)
u_dynamics = np.load(current_file_name)

preview_np_array(v_dynamics, "v_dynamics", edge_items=3)
preview_np_array(u_dynamics, "u_dynamics", edge_items=3)

v_dynamics Shape: (6000, 256).
Preview: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 3.71407560e-07 -7.73648026e-09 -2.78799703e-07 ... -6.19488685e-10
  -2.18761259e-07 -1.63569169e-08]
 [ 3.45409031e-07 -7.19492664e-09 -2.59283724e-07 ... -5.76124477e-10
  -2.03447971e-07 -1.52119327e-08]
 [ 3.21230399e-07 -6.69128178e-09 -2.41133863e-07 ... -5.35795763e-10
  -1.89206613e-07 -1.41470974e-08]]
u_dynamics Shape: (6000, 256).
Preview: [[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.

## Load the SNN Configuration and extract the fields
The SNN configuration contains:
- Numpy array with the ground_truth for each timestep
- Initial Time Offset
- Virtual Time Step Interval
- Number of Steps

In [33]:
from utils.snn import SNNSimConfig

# Load the SNN Config data
snn_config_file_name = f"{INPUT_PATH}/{band_file_name}_snn_config_{TIME_SUFFIX}.npy"

# Load the SNNConfig data as an element of the class SNNConfig
snn_config: SNNSimConfig = np.load(snn_config_file_name, allow_pickle=True).item()

# Extract the data fields from the SNNConfig object
ground_truth: np.ndarray = snn_config.ground_truth
init_offset = snn_config.init_offset
virtual_time_step_interval = snn_config.virtual_time_step_interval
num_steps = snn_config.num_steps

preview_np_array(ground_truth, "ground_truth", edge_items=3)
np.count_nonzero(ground_truth)

print("init_offset: ", init_offset)
print("virtual_time_step_interval: ", virtual_time_step_interval)
print("num_steps: ", num_steps)

ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]
init_offset:  900
virtual_time_step_interval:  1
num_steps:  6000


## Find the timesteps where the network spiked
Let's find the timesteps where the network spiked and create a `dictionary` mapping each feature neuron to the timesteps where it spiked.

In [34]:
from utils.data_analysis import find_spike_times

# Create a map storing the spike times for each feature neuron
neuron_spike_times = {}

# Call the find_spike_times util function that detects the spikes in a voltage array
spike_times_lif1 = find_spike_times(v_dynamics, u_dynamics)

total_spikes_count = 0
for (spike_time, neuron_idx) in spike_times_lif1:
    # Calculate the spike time in ms
    real_spike_time = init_offset + spike_time * virtual_time_step_interval

    # If the neuron index is not in the map (first time the feature neuron spikes), add it
    if neuron_idx not in neuron_spike_times:
        neuron_spike_times[neuron_idx] = [real_spike_time]
    # Otherwise, append the spike time to the list of spike times for that neuron
    else:
        neuron_spike_times[neuron_idx].append(real_spike_time)
    total_spikes_count += 1

    # print(f"Spike time: {real_spike_time} (iter. {spike_time}) at neuron: {neuron_idx}")

# Print the spike times for each feature neuron
print("neuron_spike_times: ", neuron_spike_times)


neuron_spike_times:  {0: [1007, 1010, 3209, 3226, 3229, 3777, 3780, 4142, 4145, 4148, 4781, 4799, 6286, 6290, 6682, 6694], 43: [1008, 4143], 1: [1009, 1017, 3228, 3781, 4146, 4149, 4788, 6291, 6685, 6695], 44: [1011], 88: [1014], 126: [1018], 13: [3222], 17: [3225], 83: [3227], 98: [3230], 50: [3232, 3785], 108: [3233], 18: [3770, 4135, 4775, 6678], 22: [3773], 8: [3776, 4141, 6679, 6692], 10: [3779, 4144], 14: [3784, 4147], 209: [3795], 29: [4138], 247: [4151], 21: [4152], 207: [4153], 201: [4777], 5: [4778, 4792], 107: [4789], 11: [6288], 2: [6289], 239: [6696]}


## Define the Time Window after the event insertion to consider as part of the event

In [35]:
# Define the time window after an HFO insertion to consider as part of the event
ripple_confidence_window = 120  # Let's give a 120ms window after the Ripple to consider as part of the event
fr_confidence_window = 60  # Let's give a 60ms window after the Fast Ripple to consider as part of the event    (TODO: Could be changed)
both_confidence_window = 120  # Let's give a 120ms window after the HFO event (Ripple or Fast Ripple) insertion to consider as part of the event

confidence_window = ripple_confidence_window
if chosen_band == MarkerType.FAST_RIPPLE:
    confidence_window = fr_confidence_window
elif chosen_band == MarkerType.BOTH:
    confidence_window = both_confidence_window

## Merge the spikes that occur at a distance less than the confidence window
In order to calculate the efficiency of the feature neurons, we need to merge the spikes that occur at a distance less than the time window. Otherwise, we would be counting the same event multiple times, which would lead to an overestimation of the efficiency.

This is **probably** safe, since each annotated event occurs at a distance greater than the considered time window, usually at least 1 order of magnitude greater.

In [36]:
# Merge the spikes that occur at a distance less than the confidence window
merged_spikes_counter = 0
for neuron_key in neuron_spike_times.keys():
    spike_times = neuron_spike_times[neuron_key]

    # Array with the merged spike times
    merged_spike_times = [spike_times[0]]

    curr_idx = 1
    comparison_idx = 0
    while curr_idx < len(spike_times):
        spike_time = spike_times[curr_idx]
        prev_spike_time = spike_times[comparison_idx]

        if spike_time - prev_spike_time < confidence_window:
            # This spike is part of the same event
            curr_idx += 1
            merged_spikes_counter += 1
            continue
        else:
            # This spike is part of a new event
            merged_spike_times.append(spike_time)
            comparison_idx = curr_idx   # Update the comparison index to the new spike time
            curr_idx += 1

    # Update the neuron_spike_times map with the merged spike times
    neuron_spike_times[neuron_key] = merged_spike_times

# Print the spike times for each feature neuron
print("Merged neuron_spike_times: ", neuron_spike_times)

print(f"Merged a total of {merged_spikes_counter} spikes out of {total_spikes_count} spikes")

Merged neuron_spike_times:  {0: [1007, 3209, 3777, 4142, 4781, 6286, 6682], 43: [1008, 4143], 1: [1009, 3228, 3781, 4146, 4788, 6291, 6685], 44: [1011], 88: [1014], 126: [1018], 13: [3222], 17: [3225], 83: [3227], 98: [3230], 50: [3232, 3785], 108: [3233], 18: [3770, 4135, 4775, 6678], 22: [3773], 8: [3776, 4141, 6679], 10: [3779, 4144], 14: [3784, 4147], 209: [3795], 29: [4138], 247: [4151], 21: [4152], 207: [4153], 201: [4777], 5: [4778], 107: [4789], 11: [6288], 2: [6289], 239: [6696]}
Merged a total of 14 spikes out of 63 spikes


## Iterate over the SNN running time and classify the feature neurons

Create a numpy array with the classification of each feature neuron.

In [37]:
from utils.snn import NeuronClass

# Create numpy array containing the class of each feature neuron
feature_neuron_class = np.full(shape=(v_dynamics.shape[1]), fill_value=NeuronClass.SILENT)
preview_np_array(feature_neuron_class, "feature_neuron_class", edge_items=3)

feature_neuron_class Shape: (256,).
Preview: [0 0 0 ... 0 0 0]


### Keep track of the relevant events each feature neuron detected

In [38]:
# Define a map to keep track of the number of relevant events each feature neuron detected
relevant_neuron_spike_times = {}

# Create an empty list for each feature neuron with spikes
for neuron_idx in neuron_spike_times.keys():
    relevant_neuron_spike_times[neuron_idx] = []

print("relevant_neuron_spike_times: ", relevant_neuron_spike_times)

relevant_neuron_spike_times:  {0: [], 43: [], 1: [], 44: [], 88: [], 126: [], 13: [], 17: [], 83: [], 98: [], 50: [], 108: [], 18: [], 22: [], 8: [], 10: [], 14: [], 209: [], 29: [], 247: [], 21: [], 207: [], 201: [], 5: [], 107: [], 11: [], 2: [], 239: []}


### Define method that searches for a relevant event in the ground_truth np array

In [39]:
# Declare a sorted np.array of the ground truth events by timestamp
ground_truth_timestamps = np.array([ann_event[1] for ann_event in ground_truth])

preview_np_array(ground_truth_timestamps, "ground_truth_timestamps", edge_items=3)

ground_truth_timestamps Shape: (199,).
Preview: [  1000.     3206.54   3770.02 ... 116096.   116769.   119000.  ]


In [40]:
def has_relevant_event(spike_time, confidence_window):
    """
    Searches for a relevant event in the ground truth array given a timestamp.
    The annotated event must be located before the spike time, since the annotation
    corresponds to the insertion of the event.
    Thus, the Event must be in the window [spike_time - confidence_window, spike_time]
    """
    for event_time in ground_truth_timestamps:
        if spike_time - confidence_window <= event_time <= spike_time:
            return True
        
        if event_time > spike_time:
            # Since the events are sorted, if the event_time is greater than the spike_time, we can stop looking
            break
    

In [41]:
# Iterate over the feature neurons that spiked
for neuron_idx in neuron_spike_times.keys():
    curr_spike_times = neuron_spike_times[neuron_idx]
    
    # For each Spike
    for curr_spike_time in curr_spike_times:
        # Since the SNN will spike after the insertion of the event, we need to consider the confidence window before the SNN spike.
        # The Annotated event is located at the insertion time (before)

        # Check if the ground truth contains at least 1 annotated event within the confidence window
        if has_relevant_event(curr_spike_time, confidence_window):
            # Add the event to the relevant neuron spike times
            relevant_neuron_spike_times[neuron_idx].append(curr_spike_time)

print("relevant_neuron_spike_times: ", relevant_neuron_spike_times)

relevant_neuron_spike_times:  {0: [1007, 3209, 3777, 4142, 4781, 6286], 43: [1008, 4143], 1: [1009, 3228, 3781, 4146, 4788, 6291], 44: [1011], 88: [1014], 126: [1018], 13: [3222], 17: [3225], 83: [3227], 98: [3230], 50: [3232, 3785], 108: [3233], 18: [4135, 4775], 22: [3773], 8: [3776, 4141], 10: [3779, 4144], 14: [3784, 4147], 209: [3795], 29: [4138], 247: [4151], 21: [4152], 207: [4153], 201: [4777], 5: [4778], 107: [4789], 11: [6288], 2: [6289], 239: [6696]}


## Statistics regarding the spiking neurons

In [42]:
# Create an array containing the relevant spike ratio for each feature neuron 
# in the order of their keys
relevant_spike_ratios = []
for neuron_idx in neuron_spike_times.keys():
    # Get all the spikes of the current neuron
    curr_spike_times = neuron_spike_times[neuron_idx]
    # Get the relevant spikes of the current neuron
    relevant_spike_times = relevant_neuron_spike_times[neuron_idx]

    # Calculate the ratio of relevant spikes
    relevant_spike_ratio = (len(relevant_spike_times) / len(curr_spike_times)) * 100 if len(curr_spike_times) > 0 else 0.0
    relevant_spike_ratios.append(relevant_spike_ratio)

In [43]:
PLOT_RELEVANCY_STATS = True

if PLOT_RELEVANCY_STATS:
    # Iterate over the feature neurons that spiked
    for iter_idx, neuron_idx in enumerate(neuron_spike_times.keys()):
        # Get all the spikes of the current neuron
        curr_spike_times = neuron_spike_times[neuron_idx]

        # Get the relevant spikes of the current neuron
        relevant_spike_times = relevant_neuron_spike_times[neuron_idx]

        print(f"Neuron {neuron_idx}: ")
        print(f"Spikes: {curr_spike_times}")
        print(f"Relevant Spikes: {relevant_spike_times}")
        print(f"Relevant Spikes Ratio: {relevant_spike_ratios[iter_idx]}%")
        print("================================================================\n")

Neuron 0: 
Spikes: [1007, 3209, 3777, 4142, 4781, 6286, 6682]
Relevant Spikes: [1007, 3209, 3777, 4142, 4781, 6286]
Relevant Spikes Ratio: 85.71428571428571%

Neuron 43: 
Spikes: [1008, 4143]
Relevant Spikes: [1008, 4143]
Relevant Spikes Ratio: 100.0%

Neuron 1: 
Spikes: [1009, 3228, 3781, 4146, 4788, 6291, 6685]
Relevant Spikes: [1009, 3228, 3781, 4146, 4788, 6291]
Relevant Spikes Ratio: 85.71428571428571%

Neuron 44: 
Spikes: [1011]
Relevant Spikes: [1011]
Relevant Spikes Ratio: 100.0%

Neuron 88: 
Spikes: [1014]
Relevant Spikes: [1014]
Relevant Spikes Ratio: 100.0%

Neuron 126: 
Spikes: [1018]
Relevant Spikes: [1018]
Relevant Spikes Ratio: 100.0%

Neuron 13: 
Spikes: [3222]
Relevant Spikes: [3222]
Relevant Spikes Ratio: 100.0%

Neuron 17: 
Spikes: [3225]
Relevant Spikes: [3225]
Relevant Spikes Ratio: 100.0%

Neuron 83: 
Spikes: [3227]
Relevant Spikes: [3227]
Relevant Spikes Ratio: 100.0%

Neuron 98: 
Spikes: [3230]
Relevant Spikes: [3230]
Relevant Spikes Ratio: 100.0%

Neuron 50: 
S

## Plot the Relevant Spike Ratio of the Feature Neurons that Spike

In [44]:
# Create a bar plot containing the % of relevant spikes for each feature neuron 
from utils.bar_plot import create_bar_fig  # Import the function to create the figure

# Define the x and y values
neuron_label = [f"Neu. {neuron_idx}" for neuron_idx in neuron_spike_times.keys()]

# sorting the bars means sorting the range factors
neurons_descending_rel_ratio = sorted(neuron_label, key=lambda x: relevant_spike_ratios[neuron_label.index(x)], reverse=True)

# Create the LIF1 Voltage
feat_neurons_relevancy_plot = create_bar_fig(
    title="Feature Neurons Relevant Spikes Ratio", 
    x_axis_label='Feature Neuron Index', 
    y_axis_label='Relevant Spike Ratio (%)',
    x=neuron_label,
    y=relevant_spike_ratios,
    x_range=neurons_descending_rel_ratio,
    sizing_mode="stretch_width",
    bar_width=0.5,
)

In [45]:
import bokeh.plotting as bplt

showPlot = True
if showPlot:
    # Show the plot
    bplt.show(feat_neurons_relevancy_plot)

In [46]:
EXPORT_RELEVANCY_PLOT = False
OUTPUT_FOLDER = "./neuron_classification/custom_subset_90-119_segment500_200"

if EXPORT_RELEVANCY_PLOT:
    file_path = f"{OUTPUT_FOLDER}/{band_file_name}_relevancy_barplot_0.07dv_time{init_offset}-{num_steps}-{virtual_time_step_interval}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="Feature Neurons Relevancy (%) Bar Plot")

    # Save the plot
    bplt.save(feat_neurons_relevancy_plot)

## Classify the Feature Neurons according to their Spiking Activity
In this step, we will update the `feature_neuron_class` array with the classification of the `Noisy Neurons`, `Ripple Neurons` and `Fast Ripple Neurons`.

We set a **threshold of 0.9** for the `Relevant Spike Ratio` to classify a neuron as a `Ripple` or `Fast Ripple` Neuron.

In [47]:
from utils.snn import marker_type_to_neuron_class

relevant_ratio_threshold = 90.0

# Iterate over the feature neurons that spiked
for iter_idx, neuron_idx in enumerate(neuron_spike_times.keys()):
    # Get all the spikes of the current neuron
    curr_spike_times = neuron_spike_times[neuron_idx]
    # Get the relevant spikes of the current neuron
    relevant_spike_times = relevant_neuron_spike_times[neuron_idx]
    # Get the relevant spikes ratio of the current neuron
    relevant_spike_ratio = relevant_spike_ratios[iter_idx]

    if relevant_spike_ratio >= relevant_ratio_threshold:
        # Classify the Neuron as a RIPPLE/FAST RIPPLE/BOTH detector based on the band we are analyzing
        feature_neuron_class[neuron_idx] = marker_type_to_neuron_class(chosen_band)
    else:
        # Classify the Neuron as Noisy
        feature_neuron_class[neuron_idx] = NeuronClass.NOISY

### Show the classification of the Feature Neurons

In [48]:
preview_np_array(feature_neuron_class, "feature_neuron_class", edge_items=3)

feature_neuron_class Shape: (256,).
Preview: [1 1 3 ... 0 0 0]
